In [1]:
# conda create -n spam_filter_env python=3.8
# conda activate spam_filter_env
# conda install pandas numpy scikit-learn nltk jupyter notebook
# python -m nltk.downloader stopwords

In [11]:
import pandas as pd
import re
import numpy as np
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

nltk.download('stopwords')
print("All libraries are successfully imported!")


All libraries are successfully imported!


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#! python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [4]:
df = pd.read_csv("email_classification.csv")
df.head()

,email,label
0,Upgrade to our premium plan for exclusive acce...,ham
1,Happy holidays from our team! Wishing you joy ...,ham
2,We're hiring! Check out our career opportuniti...,ham
3,Your Amazon account has been locked. Click her...,spam
4,Your opinion matters! Take our survey and help...,ham


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   179 non-null    object
 1   label   179 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [6]:
df.label.value_counts()

label
ham     100
spam     79
Name: count, dtype: int64

In [7]:
any_nan = df['email'].isna().any()
print(any_nan)

False


In [8]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_html(text):
    html = re.compile(r'^[^ ]<.*?>|&([a-z0-9]+|#[0-9]\"\'\“{1,6}|#x[0-9a-f]{1,6});[^A-Za-z0-9]+')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def remove_quotes(text):
    quotes = re.compile(r'[^A-Za-z0-9\s]+')
    return re.sub(quotes, '', text)


# Applying helper functions
df1 = df.copy()
df1['email'] = df1['email'].apply(lambda x: remove_URL(x))
df1['email'] = df1['email'].apply(lambda x: remove_emoji(x))
df1['email'] = df1['email'].apply(lambda x: remove_html(x))
df1['email'] = df1['email'].apply(lambda x: remove_punct(x))
df1['email'] = df1['email'].apply(lambda x: remove_quotes(x))
df1['email'] = df1['email'].str.lower()

In [9]:
df1.head()

,email,label
0,upgrade to our premium plan for exclusive acce...,ham
1,happy holidays from our team wishing you joy a...,ham
2,were hiring check out our career opportunities...,ham
3,your amazon account has been locked click here...,spam
4,your opinion matters take our survey and help ...,ham


In [10]:
df2 = df1.copy()
def clean_review_text(text):
  doc = nlp(text)
  tokens = [token.lemma_ for token in doc if not token.is_digit
            and not token.is_space]
  return " ".join(tokens)

df2['email'] = df2["email"].apply(clean_review_text)